In [ ]:
import pandas as pd
import numpy as np
import re
import joblib
import os
import datetime
import time
import csv
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, classification_report
from difflib import SequenceMatcher
from sklearn.model_selection import train_test_split

class EmailSpamFilter:
    def __init__(self, model_path='spam_model.pkl', log_path='spam_log.csv', data_path='spam_data.csv'):
        self.model_path = model_path
        self.log_path = log_path
        self.data_path = data_path
        self.blacklist_file = 'blacklist.txt'
        self.official_domains_file = 'official_domains.txt'
        self.vn_stopwords_file = 'vietnamese_stopwords.txt'

        # Tự động tải dữ liệu NLTK
        try:
            nltk.data.find('corpora/stopwords')
        except LookupError:
            nltk.download('stopwords', quiet=True)

        self.blacklist = self._load_list_from_file(self.blacklist_file, default=["spammer@rac.com", "@bad-domain.com"])
        self.official_domains = self._load_list_from_file(self.official_domains_file, default=["vju.ac.vn", "st.vju.ac.vn", "gmail.com"])

        self.stopwords = set(nltk.corpus.stopwords.words('english'))
        vn_stops = self._load_list_from_file(self.vn_stopwords_file, default=["là", "thì", "mà", "bị", "bởi", "của", "được", "tại", "vì"])
        self.stopwords.update(vn_stops)

        self.label_mapping = {'ham': 0, 'spam': 1}
        self.reverse_label_mapping = {0: 'ham', 1: 'spam'}

        # 1. min_df=1: QUAN TRỌNG. Chấp nhận từ xuất hiện 1 lần (để chạy được data mẫu 4 dòng).
        # 2. ngram_range=(1, 1): Chỉ học từ đơn. Giúp tránh Overfitting với dữ liệu nhỏ/vừa.
        # 3. max_features: Giới hạn số lượng từ vựng AI được phép nhớ.
        self.preprocessor = ColumnTransformer(
            transformers=[
                # Sender: Chỉ nhớ 500 từ sender quan trọng nhất
                ('sender', TfidfVectorizer(ngram_range=(1, 1), token_pattern=r'\b\w+\b', min_df=1, max_features=500), 'sender'),
                # Subject: Chỉ nhớ 500 từ tiêu đề quan trọng nhất
                ('subject', TfidfVectorizer(ngram_range=(1, 1), token_pattern=r'\b\w+\b', min_df=1, max_features=500), 'subject'),
                # Content: Chỉ nhớ 3000 từ nội dung quan trọng nhất
                ('content', TfidfVectorizer(ngram_range=(1, 1), token_pattern=r'\b\w+\b', min_df=1, max_features=3000), 'content')
            ],
            remainder='drop'
        )

        self.model = make_pipeline(self.preprocessor, MultinomialNB())
        self.is_trained = False
        self.global_content_history = []
        self._load_model()
        self._init_log_file()

    def _init_log_file(self):
        if not os.path.exists(self.log_path):
            with open(self.log_path, 'w', newline='', encoding='utf-8-sig') as f:
                writer = csv.writer(f)
                writer.writerow(["Timestamp", "Sender", "Subject", "Result", "Reason", "ProcessingTime(s)"])

    def _load_list_from_file(self, filepath, default):
        if not os.path.exists(filepath):
            with open(filepath, 'w', encoding='utf-8') as f:
                f.write("\n".join(default))
            return set(default)
        with open(filepath, 'r', encoding='utf-8') as f:
            return {line.strip().lower() for line in f if line.strip()}

    def get_clean_email(self, sender):
        sender = str(sender).lower().strip()
        match = re.search(r'[\w\.\-\+]+@[\w\.\-]+', sender)
        if match: return match.group(0)
        return sender

    def get_domain(self, email):
        if "@" in email: return email.split("@")[-1]
        return None

    def check_spoofing(self, sender_email):
        sender_domain = self.get_domain(sender_email)
        if not sender_domain: return False, None
        if sender_domain in self.official_domains: return False, None
        for official in self.official_domains:
            if sender_domain.endswith("." + official): return False, None

        for official in self.official_domains:
            similarity = SequenceMatcher(None, sender_domain, official).ratio()
            if 0.8 < similarity < 1.0:
                return True, f"Nghi vấn giả mạo domain '{official}'"
        return False, None

    def check_blacklist(self, email):
        for blocked_item in self.blacklist:
            if blocked_item.startswith("@"):
                if email.endswith(blocked_item): return True
            elif email == blocked_item: return True
        return False

    def remove_stopwords(self, text):
        words = text.split()
        clean_words = [w for w in words if w not in self.stopwords]
        return " ".join(clean_words)

    def clean_text(self, text):
        original_text = str(text)
        if not original_text.strip(): return "empty_content_placeholder"
        cleaned_text = original_text.lower()
        cleaned_text = re.sub(r'<[a-z/][^>]*>', '', cleaned_text)
        cleaned_text = re.sub(r'[^\w\s]', ' ', cleaned_text, flags=re.UNICODE)
        cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()
        cleaned_text = self.remove_stopwords(cleaned_text)
        if not cleaned_text: return "empty_content_placeholder"
        return cleaned_text

    def _prepare_dataframe(self, df):
        processed_df = pd.DataFrame()
        clean_senders = df['sender'].apply(self.get_clean_email)
        processed_df['sender'] = clean_senders.apply(self.clean_text)
        processed_df['subject'] = df['subject'].apply(self.clean_text)
        processed_df['content'] = df['content'].apply(self.clean_text)
        return processed_df

    def log_result(self, sender, subject, result, reason, proc_time):
        ts = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        with open(self.log_path, "a", newline='', encoding='utf-8-sig') as f:
            writer = csv.writer(f)
            writer.writerow([ts, sender, subject, result.upper(), reason, f"{proc_time:.4f}"])

    def train(self, df):
        print("--- Đang huấn luyện mô hình (Re-training) ---")
        if 'text' in df.columns and 'content' not in df.columns:
            df = df.rename(columns={'text': 'content'})

        required_cols = ['sender', 'subject', 'content', 'label']
        for col in required_cols:
            if col not in df.columns: df[col] = ""
        df = df.fillna('')

        # Lưu đè dữ liệu mới nhất vào file data
        df[required_cols].to_csv(self.data_path, index=False, encoding='utf-8-sig')

        X = self._prepare_dataframe(df)
        y = df['label']
        y_encoded = y.map(self.label_mapping)

        # Loại bỏ nhãn lỗi
        if y_encoded.isnull().any():
            nan_idx = y_encoded[y_encoded.isnull()].index
            X = X.drop(nan_idx)
            y_encoded = y_encoded.drop(nan_idx)

        # Nếu dữ liệu ít thì train hết, không split
        if len(y_encoded) < 10:
             self.model.fit(X, y_encoded)
        else:
            X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)
            self.model.fit(X_train, y_train)
            # In báo cáo nhanh
            acc = accuracy_score(y_test, self.model.predict(X_test))
            print(f"-> Độ chính xác mô hình: {acc*100:.2f}%")

        self.is_trained = True
        joblib.dump(self.model, self.model_path)
        print("-> Đã lưu mô hình mới.")

    # --- YÊU CẦU 4: SỬA LỖI VÀ GHI NHỚ ---
    def teach_ai(self, sender, subject, content, correct_label):
        print(f"\n--- FEEDBACK: Đang dạy lại AI rằng mail này là '{correct_label.upper()}' ---")

        # Đọc dữ liệu cũ
        if os.path.exists(self.data_path):
            df = pd.read_csv(self.data_path, encoding='utf-8-sig')
            df = df.fillna('')
        else:
            df = pd.DataFrame(columns=['sender', 'subject', 'content', 'label'])

        # Thêm dữ liệu mới
        new_row = pd.DataFrame({'sender': [sender], 'subject': [subject], 'content': [content], 'label': [correct_label]})
        df = pd.concat([df, new_row], ignore_index=True)

        # Train lại ngay lập tức
        self.train(df)
        print("-> AI đã học xong! Lần sau gặp mail này nó sẽ đoán đúng.")

    def classify(self, sender, subject, content):
        start_time = time.time()

        # Logic check cơ bản
        if not content or not str(content).strip():
            return self._finalize(sender, subject, "spam", "Lỗi: Nội dung rỗng", start_time)

        clean_sender = self.get_clean_email(sender)
        if self.check_blacklist(clean_sender):
            return self._finalize(sender, subject, "spam", "Chặn: Blacklist", start_time)

        if any(SequenceMatcher(None, content, old).ratio() >= 0.85 for old in self.global_content_history):
            return self._finalize(sender, subject, "spam", "Chặn: Spam lặp lại", start_time)
        self.global_content_history.append(content)
        if len(self.global_content_history) > 50: self.global_content_history.pop(0)

        if not self.is_trained: return "unknown", "Lỗi: Model chưa train"

        # AI Phân loại
        input_df = pd.DataFrame({'sender': [sender], 'subject': [subject], 'content': [content]})
        X_input = self._prepare_dataframe(input_df)

        try:
            pred_num = self.model.predict(X_input)[0]
            prob = self.model.predict_proba(X_input).max()
            pred = self.reverse_label_mapping.get(pred_num, "unknown")
        except:
            return "error", "Lỗi phân loại"

        reason = f"AI Score: {prob:.2f}"

        # Check giả mạo
        is_spoof, spoof_msg = self.check_spoofing(clean_sender)
        if is_spoof:
            if pred == "ham":
                pred = "warning"
                reason += f" | CẢNH BÁO: {spoof_msg}"
            else:
                reason += f" | {spoof_msg}"

        return self._finalize(sender, subject, pred, reason, start_time)

    def _finalize(self, sender, subject, result, reason, start_time):
        proc_time = time.time() - start_time
        self.log_result(sender, subject, result, reason, proc_time)
        return result, reason

    def _load_model(self):
        if os.path.exists(self.model_path):
            self.model = joblib.load(self.model_path)
            self.is_trained = True

# --- PHẦN CHẠY CHÍNH (MAIN) ---
if __name__ == "__main__":
    filter_sys = EmailSpamFilter()

    # 1. Khởi tạo dữ liệu train nếu chưa có
    if not os.path.exists(filter_sys.data_path):
        print("-> Tạo dữ liệu mẫu...")
        data = {
            'sender': ["hr@company.com", "sale@shopee-fake.com", "vayvon@credit.vn", "student@vju.ac.vn"],
            'subject': ["Họp team", "SALE SẬP SÀN", "Vay vốn nhanh", "Nộp bài tập"],
            'content': ["Họp 9h nhé", "Mua ngay kẻo lỡ", "Lãi suất 0%", "Em gửi bài ạ"],
            'label': ["ham", "spam", "spam", "ham"]
        }
        filter_sys.train(pd.DataFrame(data))
    else:
        # Load dữ liệu cũ lên để đảm bảo model hoạt động
        filter_sys.train(pd.read_csv(filter_sys.data_path, encoding='utf-8-sig'))

    # --- YÊU CẦU 1 & 3: QUÉT TỰ ĐỘNG TỪ FILE CSV ---
    input_file = 'input_emails.csv'

    # Tạo file input mẫu nếu chưa có để người dùng biết đường điền
    if not os.path.exists(input_file):
        print(f"\n[THÔNG BÁO] Không tìm thấy file '{input_file}'.")
        print("-> Đang tạo file mẫu. Hãy mở file này lên, điền email cần lọc vào rồi chạy lại code.")
        sample_input = pd.DataFrame({
            'sender': ['test@gmail.com', 'hacker@vju.net'],
            'subject': ['Test Subject', 'Fake Subject'],
            'content': ['Hello world', 'Click here to win']
        })
        sample_input.to_csv(input_file, index=False, encoding='utf-8-sig')

    else:
        print(f"\n--- BẮT ĐẦU QUÉT FILE '{input_file}' ---")
        df_input = pd.read_csv(input_file, encoding='utf-8-sig')
        df_input = df_input.fillna('')

        count = 0
        for index, row in df_input.iterrows():
            count += 1
            res, reason = filter_sys.classify(row['sender'], row['subject'], row['content'])
            print(f"[{count}] {row['sender']} -> {res.upper()} ({reason})")

        print(f"\n-> Đã quét xong {count} email. Kiểm tra file 'spam_log.csv' để xem lịch sử.")

    # --- YÊU CẦU 4: MENU TƯƠNG TÁC SỬA SAI ---
    while True:
        print("\n" + "="*40)
        print("MENU QUẢN LÝ (Gõ số để chọn):")
        print("1. Kiểm tra nhanh 1 email")
        print("2. Báo cáo AI sai (Dạy lại AI)")
        print("3. Thoát")
        choice = input("Lựa chọn: ")

        if choice == '1':
            s = input("Sender: ")
            sub = input("Subject: ")
            c = input("Content: ")
            res, reason = filter_sys.classify(s, sub, c)
            print(f"-> KẾT QUẢ: {res.upper()} | {reason}")

        elif choice == '2':
            print("\n--- SỬA LỖI AI (FEEDBACK) ---")
            print("Hãy nhập thông tin email mà AI đã đoán sai:")
            s = input("Sender: ")
            sub = input("Subject: ")
            c = input("Content: ")
            lbl = input("Nhãn ĐÚNG là gì (ham/spam)? ").lower().strip()

            if lbl in ['ham', 'spam']:
                filter_sys.teach_ai(s, sub, c, lbl)
            else:
                print("Lỗi: Nhãn phải là 'ham' hoặc 'spam'.")

        elif choice == '3':
            print("Đã thoát.")
            break

--- Đang huấn luyện mô hình (Re-training) ---
-> Độ chính xác mô hình: 96.32%
-> Đã lưu mô hình mới.

--- BẮT ĐẦU QUÉT FILE 'input_emails.csv' ---
[1] nguyen.van.an@st.vju.ac.vn -> HAM (AI Score: 1.00)
[2] admin@nha-cai-uy-tin.vip -> SPAM (AI Score: 1.00)
[3] hr.recruitment@shopee.vn -> HAM (AI Score: 1.00)
[4] hotro@vay-tien-online-nhanh.com -> SPAM (AI Score: 1.00)
[5] le.thi.mai@gmail.com -> HAM (AI Score: 1.00)
[6] cskh@momo-qua-tang.xyz -> SPAM (AI Score: 1.00)
[7] tran.hoang.nam@tech.com.vn -> HAM (AI Score: 1.00)
[8] admin@dating-tinh-mot-dem.net -> SPAM (AI Score: 0.98)
[9] billing@vinaphone.vn -> HAM (AI Score: 0.99)
[10] sales@bat-dong-san-hung-thinh.com -> SPAM (AI Score: 1.00)
[11] nguyen.thu.huong@vju.ac.vn -> HAM (AI Score: 1.00)
[12] support@facebook-security-check.com -> SPAM (AI Score: 1.00)
[13] pham.minh.tuan@outlook.com -> HAM (AI Score: 1.00)
[14] tuyendung@viec-lam-luong-cao.info -> SPAM (AI Score: 1.00)
[15] admin@chung-khoan-quoc-te.org -> SPAM (AI Score: 0.97)
